In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = "notMNIST.pickle"
with open(pickle_file, "rb") as fp:
    all_data = pickle.load(fp)
    train_dataset = all_data["train_dataset"]
    train_labels = all_data["train_labels"]
    validation_dataset = all_data["valid_dataset"]
    validation_labels = all_data["valid_labels"]
    test_dataset = all_data["test_dataset"]
    test_labels = all_data["test_labels"]
    del all_data
    print("Training set: {} {}".format(train_dataset.shape, train_labels.shape))
    print("Validation set: {} {}".format(validation_dataset.shape, validation_labels.shape))
    print("Test set: {} {}".format(test_dataset.shape, test_labels.shape))

Training set: (200000, 28, 28) (200000,)
Validation set: (10000, 28, 28) (10000,)
Test set: (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1

def reformat(dataset, labels):
    dataset = dataset.reshape(-1, image_size, image_size, num_channels)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
validation_dataset, validation_labels = reformat(validation_dataset, validation_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print("Training set: {} {}".format(train_dataset.shape, train_labels.shape))
print("Validation set: {} {}".format(validation_dataset.shape, validation_labels.shape))
print("Test set: {} {}".format(test_dataset.shape, test_labels.shape))

Training set: (200000, 28, 28, 1) (200000, 10)
Validation set: (10000, 28, 28, 1) (10000, 10)
Test set: (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    num_hits = np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
    return 100.0 * num_hits / predictions.shape[0]

In [47]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_validation_dataset = tf.constant(validation_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    weights_2 = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    size_post_2_poolings = (((image_size - 4) // 2) - 4) // 2
    weights_3 = tf.Variable(tf.truncated_normal(
        [size_post_2_poolings * size_post_2_poolings * depth, num_hidden], stddev=0.1))
    weights_4 = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    
    bias_1 = tf.Variable(tf.zeros([depth]))
    bias_2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    bias_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    bias_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data):
        conv_1 = tf.nn.conv2d(data, weights_1, [1, 1, 1, 1], padding="VALID")
        relu_1 = tf.nn.relu(conv_1 + bias_1)
        pool_1 = tf.nn.max_pool(relu_1, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")
        
        conv_2 = tf.nn.conv2d(pool_1, weights_2, [1, 1, 1, 1], padding="VALID")
        relu_2 = tf.nn.relu(conv_2 + bias_2)
        pool_2 = tf.nn.max_pool(relu_2, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")
        
        shape = pool_2.get_shape().as_list()
        reshaped_pool_2 = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        relu_3 = tf.nn.relu(tf.matmul(reshaped_pool_2, weights_3) + bias_3)
        return tf.matmul(relu_3, weights_4) + bias_4
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                          labels=tf_train_labels, logits=logits))
    
    optimiser = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    validation_prediction = tf.nn.softmax(model(tf_validation_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))        
    

In [48]:
num_steps = 2001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialised")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        
        _, loss_step, prediction_step = session.run([optimiser, loss, train_prediction],
                                                   feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch Loss at step {}: {}".format(step, loss_step))
            print("Minibatch Training Accuracy: {}%".format(accuracy(prediction_step, batch_labels)))
            print("Minibatch Validation Accuracy: {}%".format(accuracy(validation_prediction.eval(), validation_labels)))

    print("Test Accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))        


Initialised
Minibatch Loss at step 0: 2.73628306388855
Minibatch Training Accuracy: 12.5%
Minibatch Validation Accuracy: 10.0%
Minibatch Loss at step 500: 0.5637091398239136
Minibatch Training Accuracy: 87.5%
Minibatch Validation Accuracy: 80.58%
Minibatch Loss at step 1000: 0.33233723044395447
Minibatch Training Accuracy: 93.75%
Minibatch Validation Accuracy: 82.89%
Minibatch Loss at step 1500: 0.5805478692054749
Minibatch Training Accuracy: 81.25%
Minibatch Validation Accuracy: 84.42%
Minibatch Loss at step 2000: 0.09617877006530762
Minibatch Training Accuracy: 100.0%
Minibatch Validation Accuracy: 85.38%
Test Accuracy: 91.59
